# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Finding Donors for *CharityML*

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Please specify WHICH VERSION OF PYTHON you are using when submitting this notebook. Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

## Getting Started

In this project, you will employ several supervised algorithms of your choice to accurately model individuals' income using data collected from the 1994 U.S. Census. You will then choose the best candidate algorithm from preliminary results and further optimize this algorithm to best model the data. Your goal with this implementation is to construct a model that accurately predicts whether an individual makes more than $50,000. This sort of task can arise in a non-profit setting, where organizations survive on donations.  Understanding an individual's income can help a non-profit better understand how large of a donation to request, or whether or not they should reach out to begin with.  While it can be difficult to determine an individual's general income bracket directly from public sources, we can (as we will see) infer this value from other publically available features. 

The dataset for this project originates from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/Census+Income). The datset was donated by Ron Kohavi and Barry Becker, after being published in the article _"Scaling Up the Accuracy of Naive-Bayes Classifiers: A Decision-Tree Hybrid"_. You can find the article by Ron Kohavi [online](https://www.aaai.org/Papers/KDD/1996/KDD96-033.pdf). The data we investigate here consists of small changes to the original dataset, such as removing the `'fnlwgt'` feature and records with missing or ill-formatted entries.

----
## Exploring the Data
Run the code cell below to load necessary Python libraries and load the census data. Note that the last column from this dataset, `'income'`, will be our target label (whether an individual makes more than, or at most, $50,000 annually). All other columns are features about each individual in the census database.

In [2]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for DataFrames

# Import supplementary visualization code visuals.py
import visuals as vs
from visuals import distribution
# Pretty display for notebooks
%matplotlib inline

# Load the Census dataset
data = pd.read_csv(r"C:\Users\habib\OneDrive\المستندات\Depi_workingSpace\Depi_workigSpace\Contents\ML\Project_2\census.csv")


# Success - Display the first record
display(data.head())

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K



<div>
    <button onclick="like()" style="padding: 10px 20px; margin-right: 10px;">👍 Like</button>
   
</div>



### Implementation: Data Exploration
A cursory investigation of the dataset will determine how many individuals fit into either group, and will tell us about the percentage of these individuals making more than \$50,000. In the code cell below, you will need to compute the following:
- The total number of records, `'n_records'`
- The number of individuals making more than \$50,000 annually, `'n_greater_50k'`.
- The number of individuals making at most \$50,000 annually, `'n_at_most_50k'`.
- The percentage of individuals making more than \$50,000 annually, `'greater_percent'`.

** HINT: ** You may need to look at the table above to understand how the `'income'` entries are formatted. 

In [3]:
# TODO: Total number of records
n_records =  data.shape[0]

# TODO: Number of records where individual's income is more than $50,000
n_greater_50k = data[data['income'] == '>50K'].shape[0]

# TODO: Number of records where individual's income is at most $50,000
n_at_most_50k = data[data['income'] == '<=50K'].shape[0]

# TODO: Percentage of individuals whose income is more than $50,000
greater_percent = round((n_greater_50k / n_records) * 100 , 2)

# Print the results
print("Total number of records: {}".format(n_records))
print("Individuals making more than $50,000: {}".format(n_greater_50k))
print("Individuals making at most $50,000: {}".format(n_at_most_50k))
print("Percentage of individuals making more than $50,000: {}%".format(greater_percent))

Total number of records: 45222
Individuals making more than $50,000: 11208
Individuals making at most $50,000: 34014
Percentage of individuals making more than $50,000: 24.78%


<div>
    <button onclick="dislike()" style="padding: 10px 20px;">👎 Dislike</button>

</div>

### where your output code here

** Featureset Exploration **

* **age**: continuous. 
* **workclass**: Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked. 
* **education**: Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool. 
* **education-num**: continuous. 
* **marital-status**: Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse. 
* **occupation**: Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces. 
* **relationship**: Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried. 
* **race**: Black, White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other. 
* **sex**: Female, Male. 
* **capital-gain**: continuous. 
* **capital-loss**: continuous. 
* **hours-per-week**: continuous. 
* **native-country**: United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

----
## Preparing the Data
Before data can be used as input for machine learning algorithms, it often must be cleaned, formatted, and restructured — this is typically known as **preprocessing**. Fortunately, for this dataset, there are no invalid or missing entries we must deal with, however, there are some qualities about certain features that must be adjusted. This preprocessing can help tremendously with the outcome and predictive power of nearly all learning algorithms.

### Transforming Skewed Continuous Features
A dataset may sometimes contain at least one feature whose values tend to lie near a single number, but will also have a non-trivial number of vastly larger or smaller values than that single number.  Algorithms can be sensitive to such distributions of values and can underperform if the range is not properly normalized. With the census dataset two features fit this description: '`capital-gain'` and `'capital-loss'`. 

Run the code cell below to plot a histogram of these two features. Note the range of the values present and how they are distributed.

In [4]:
import plotly.express as px
import pandas as pd
import numpy as np
from IPython.display import display

income_raw = data['income']
features_raw = data.drop('income', axis = 1)


skewed_features = ['capital-gain', 'capital-loss']
plot_data = data[skewed_features]


for feature in skewed_features:
    fig = px.histogram(
        plot_data, 
        x=feature,
        nbins=100, # A high number of bins to show detail
        title=f'Distribution of {feature.replace("-", " ").title()} (Original Data)',
        color_discrete_sequence=['#4C78A8'] 
    )
    
    # Customize layout to emphasize the vast difference in value counts
    fig.update_layout(
        bargap=0.01,
        title_x=0.5, # Center the title
        yaxis_title="Count of Records",
        xaxis_title=feature.replace("-", " ").title(),
        showlegend=False,
        plot_bgcolor='white', # Clean background
        hovermode="x unified" # Interactive hover
    )
    
    # Set x-axis type to linear for true representation, but emphasize the 0 bar
    fig.update_xaxes(
        showgrid=True, 
        gridcolor='lightgray', 
        zeroline=True, 
        zerolinecolor='gray'
    )
    fig.update_yaxes(
        showgrid=True, 
        gridcolor='lightgray'
    )
    
    display(fig)

For highly-skewed feature distributions such as `'capital-gain'` and `'capital-loss'`, it is common practice to apply a <a href="https://en.wikipedia.org/wiki/Data_transformation_(statistics)">logarithmic transformation</a> on the data so that the very large and very small values do not negatively affect the performance of a learning algorithm. Using a logarithmic transformation significantly reduces the range of values caused by outliers. Care must be taken when applying this transformation however: The logarithm of `0` is undefined, so we must translate the values by a small amount above `0` to apply the the logarithm successfully.

Run the code cell below to perform a transformation on the data and visualize the results. Again, note the range of values and how they are distributed. 


<div>
    <button onclick="like()" style="padding: 10px 20px; margin-right: 10px;">👍 Like</button>
   
</div>

In [5]:
import pandas as pd
import numpy as np
import plotly.express as px
from IPython.display import display

# --- ASSUMPTION: 'data' is the loaded census DataFrame from earlier steps ---
# Based on the provided code structure:
income_raw = data['income']
features_raw = data.drop('income', axis = 1)

# Log-transform the skewed features
skewed = ['capital-gain', 'capital-loss']
features_log_transformed = features_raw.copy()
# Apply np.log(x + 1) to avoid log(0)
features_log_transformed[skewed] = features_raw[skewed].apply(lambda x: np.log(x + 1))

# --- Visualization of Log-Transformed Features ---

# Use the log-transformed data for visualization
plot_data_log = features_log_transformed[skewed]

for feature in skewed:
    # Use Plotly Express to visualize the transformed distribution
    fig = px.histogram(
        plot_data_log, 
        x=feature,
        nbins=50, # Fewer bins may be appropriate for the compressed range
        title=f'Distribution of {feature.replace("-", " ").title()} (LOG-TRANSFORMED)',
        color_discrete_sequence=['#CC3366'] # A new color to distinguish from the original plot
    )
    
    # Customize layout to reflect the transformed data
    fig.update_layout(
        bargap=0.01,
        title_x=0.5, # Center the title
        yaxis_title="Count of Records",
        xaxis_title=f"{feature.replace('-', ' ').title()} (Log Transformed)",
        showlegend=False,
        plot_bgcolor='white', # Clean background
        hovermode="x unified" # Interactive hover
    )
    
    # Update axes to show the new compressed range
    fig.update_xaxes(
        showgrid=True, 
        gridcolor='lightgray', 
        zeroline=True, 
        zerolinecolor='gray'
    )
    fig.update_yaxes(
        showgrid=True, 
        gridcolor='lightgray'
    )
    
    display(fig)

display(features_log_transformed.head())

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,7.684784,0.0,40.0,United-States
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,13.0,United-States
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.000000,0.0,40.0,United-States
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.000000,0.0,40.0,United-States
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.000000,0.0,40.0,Cuba


### Normalizing Numerical Features
In addition to performing transformations on features that are highly skewed, it is often good practice to perform some type of scaling on numerical features. Applying a scaling to the data does not change the shape of each feature's distribution (such as `'capital-gain'` or `'capital-loss'` above); however, normalization ensures that each feature is treated equally when applying supervised learners. Note that once scaling is applied, observing the data in its raw form will no longer have the same original meaning, as exampled below.

Run the code cell below to normalize each numerical feature. We will use [`sklearn.preprocessing.MinMaxScaler`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html) for this.

In [6]:
# Import sklearn.preprocessing.StandardScaler
from sklearn.preprocessing import MinMaxScaler

# Initialize a scaler, then apply it to the features
scaler = MinMaxScaler() # default=(0, 1)
numerical = ['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']

features_log_minmax_transform = pd.DataFrame(data = features_log_transformed)
features_log_minmax_transform[numerical] = scaler.fit_transform(features_log_transformed[numerical])

# Show an example of a record with scaling applied
display(features_log_minmax_transform.head(n = 5))


,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
0,0.301370,State-gov,Bachelors,0.800000,Never-married,Adm-clerical,Not-in-family,White,Male,0.667492,0.0,0.397959,United-States
1,0.452055,Self-emp-not-inc,Bachelors,0.800000,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.000000,0.0,0.122449,United-States
2,0.287671,Private,HS-grad,0.533333,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.000000,0.0,0.397959,United-States
3,0.493151,Private,11th,0.400000,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.000000,0.0,0.397959,United-States
4,0.150685,Private,Bachelors,0.800000,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.000000,0.0,0.397959,Cuba



<div>
    <button onclick="like()" style="padding: 10px 20px; margin-right: 10px;">👍 Like</button>
   
</div>

### Implementation: Data Preprocessing

From the table in **Exploring the Data** above, we can see there are several features for each record that are non-numeric. Typically, learning algorithms expect input to be numeric, which requires that non-numeric features (called *categorical variables*) be converted. One popular way to convert categorical variables is by using the **one-hot encoding** scheme. One-hot encoding creates a _"dummy"_ variable for each possible category of each non-numeric feature. For example, assume `someFeature` has three possible entries: `A`, `B`, or `C`. We then encode this feature into `someFeature_A`, `someFeature_B` and `someFeature_C`.

|   | someFeature |                    | someFeature_A | someFeature_B | someFeature_C |
| :-: | :-: |                            | :-: | :-: | :-: |
| 0 |  B  |  | 0 | 1 | 0 |
| 1 |  C  | ----> one-hot encode ----> | 0 | 0 | 1 |
| 2 |  A  |  | 1 | 0 | 0 |

Additionally, as with the non-numeric features, we need to convert the non-numeric target label, `'income'` to numerical values for the learning algorithm to work. Since there are only two possible categories for this label ("<=50K" and ">50K"), we can avoid using one-hot encoding and simply encode these two categories as `0` and `1`, respectively. In code cell below, you will need to implement the following:
 - Use [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) to perform one-hot encoding on the `'features_log_minmax_transform'` data.
 - Convert the target label `'income_raw'` to numerical entries.
   - Set records with "<=50K" to `0` and records with ">50K" to `1`.

In [7]:
# TODO: One-hot encode the 'features_log_minmax_transform' data using pandas.get_dummies()
features_final = pd.get_dummies(features_log_minmax_transform)

# TODO: Encode the 'income_raw' data to numerical values
income = income_raw.map({'<=50K': 0, '>50K': 1})

# Print the number of features after one-hot encoding
encoded = list(features_final.columns)
print("{} total features after one-hot encoding.".format(len(encoded)))

# Uncomment the following line to see the encoded feature names
print(encoded)

103 total features after one-hot encoding.
['age', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week', 'workclass_ Federal-gov', 'workclass_ Local-gov', 'workclass_ Private', 'workclass_ Self-emp-inc', 'workclass_ Self-emp-not-inc', 'workclass_ State-gov', 'workclass_ Without-pay', 'education_level_ 10th', 'education_level_ 11th', 'education_level_ 12th', 'education_level_ 1st-4th', 'education_level_ 5th-6th', 'education_level_ 7th-8th', 'education_level_ 9th', 'education_level_ Assoc-acdm', 'education_level_ Assoc-voc', 'education_level_ Bachelors', 'education_level_ Doctorate', 'education_level_ HS-grad', 'education_level_ Masters', 'education_level_ Preschool', 'education_level_ Prof-school', 'education_level_ Some-college', 'marital-status_ Divorced', 'marital-status_ Married-AF-spouse', 'marital-status_ Married-civ-spouse', 'marital-status_ Married-spouse-absent', 'marital-status_ Never-married', 'marital-status_ Separated', 'marital-status_ Widowed', 'occupation_ A

<div>
    <button onclick="dislike()" style="padding: 10px 20px;">👎 Dislike</button>

</div>

### Shuffle and Split Data
Now all _categorical variables_ have been converted into numerical features, and all numerical features have been normalized. As always, we will now split the data (both features and their labels) into training and test sets. 80% of the data will be used for training and 20% for testing.

Run the code cell below to perform this split.

In [8]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features_final, 
                                                    income, 
                                                    test_size = 0.2, 
                                                    random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_test.shape[0]))

Training set has 36177 samples.
Testing set has 9045 samples.


<div>
    <button onclick="dislike()" style="padding: 10px 20px;">👎 Dislike</button>

</div>

----
## Evaluating Model Performance
In this section, we will investigate four different algorithms, and determine which is best at modeling the data. Three of these algorithms will be supervised learners of your choice, and the fourth algorithm is known as a *naive predictor*.

### Metrics and the Naive Predictor
*CharityML*, equipped with their research, knows individuals that make more than \$50,000 are most likely to donate to their charity. Because of this, *CharityML* is particularly interested in predicting who makes more than \$50,000 accurately. It would seem that using **accuracy** as a metric for evaluating a particular model's performace would be appropriate. Additionally, identifying someone that *does not* make more than \$50,000 as someone who does would be detrimental to *CharityML*, since they are looking to find individuals willing to donate. Therefore, a model's ability to precisely predict those that make more than \$50,000 is *more important* than the model's ability to **recall** those individuals. We can use **F-beta score** as a metric that considers both precision and recall:

$$ F_{\beta} = (1 + \beta^2) \cdot \frac{precision \cdot recall}{\left( \beta^2 \cdot precision \right) + recall} $$

In particular, when $\beta = 0.5$, more emphasis is placed on precision. This is called the **F$_{0.5}$ score** (or F-score for simplicity).

Looking at the distribution of classes (those who make at most \$50,000, and those who make more), it's clear most individuals do not make more than \$50,000. This can greatly affect **accuracy**, since we could simply say *"this person does not make more than \$50,000"* and generally be right, without ever looking at the data! Making such a statement would be called **naive**, since we have not considered any information to substantiate the claim. It is always important to consider the *naive prediction* for your data, to help establish a benchmark for whether a model is performing well. That been said, using that prediction would be pointless: If we predicted all people made less than \$50,000, *CharityML* would identify no one as donors. 


#### Note: Recap of accuracy, precision, recall

** Accuracy ** measures how often the classifier makes the correct prediction. It’s the ratio of the number of correct predictions to the total number of predictions (the number of test data points).

** Precision ** tells us what proportion of messages we classified as spam, actually were spam.
It is a ratio of true positives(words classified as spam, and which are actually spam) to all positives(all words classified as spam, irrespective of whether that was the correct classificatio), in other words it is the ratio of

`[True Positives/(True Positives + False Positives)]`

** Recall(sensitivity)** tells us what proportion of messages that actually were spam were classified by us as spam.
It is a ratio of true positives(words classified as spam, and which are actually spam) to all the words that were actually spam, in other words it is the ratio of

`[True Positives/(True Positives + False Negatives)]`

For classification problems that are skewed in their classification distributions like in our case, for example if we had a 100 text messages and only 2 were spam and the rest 98 weren't, accuracy by itself is not a very good metric. We could classify 90 messages as not spam(including the 2 that were spam but we classify them as not spam, hence they would be false negatives) and 10 as spam(all 10 false positives) and still get a reasonably good accuracy score. For such cases, precision and recall come in very handy. These two metrics can be combined to get the F1 score, which is weighted average(harmonic mean) of the precision and recall scores. This score can range from 0 to 1, with 1 being the best possible F1 score(we take the harmonic mean as we are dealing with ratios).

### Question 1 - Naive Predictor Performace
* If we chose a model that always predicted an individual made more than $50,000, what would  that model's accuracy and F-score be on this dataset? You must use the code cell below and assign your results to `'accuracy'` and `'fscore'` to be used later.

** Please note ** that the purpose of generating a naive predictor is simply to show what a base model without any intelligence would look like. In the real world, ideally your base model would be either the results of a previous model or could be based on a research paper upon which you are looking to improve. When there is no benchmark model set, getting a result better than random choice is a place you could start from.

** HINT: ** 

* When we have a model that always predicts '1' (i.e. the individual makes more than 50k) then our model will have no True Negatives(TN) or False Negatives(FN) as we are not making any negative('0' value) predictions. Therefore our Accuracy in this case becomes the same as our Precision(True Positives/(True Positives + False Positives)) as every prediction that we have made with value '1' that should have '0' becomes a False Positive; therefore our denominator in this case is the total number of records we have in total. 
* Our Recall score(True Positives/(True Positives + False Negatives)) in this setting becomes 1 as we have no False Negatives.

In [9]:
import numpy as np

# Naive case: always predict '>50K' (1)
TP = np.sum(income)  # True Positives — all actual '>50K'
FP = income.count() - TP  # False Positives — all '<=50K' predicted as '>50K'

TN = 0  # No predicted negatives
FN = 0  # No predicted negatives

# Calculate accuracy, precision, and recall
accuracy = TP / income.count()
precision = TP / (TP + FP)
recall = TP / (TP + FN)  # FN = 0 here, but we keep it for clarity

# Calculate F-score for beta = 0.5
beta = 0.5
fscore = (1 + beta**2) * (precision * recall) / ((beta**2 * precision) + recall)

# Print the results
print("Naive Predictor: [Accuracy score: {:.4f}, F-score: {:.4f}]".format(accuracy, fscore))

Naive Predictor: [Accuracy score: 0.2478, F-score: 0.2917]


###  Supervised Learning Models
**The following are some of the supervised learning models that are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent Classifier (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
List three of the supervised learning models above that are appropriate for this problem that you will test on the census data. For each model chosen

- Describe one real-world application in industry where the model can be applied. 
- What are the strengths of the model; when does it perform well?
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

** HINT: **

Structure your answer in the same format as above^, with 4 parts for each of the three models you pick. Please include references with your answer.

**Answer: **
>> `Model 1: AdaBoost Classifier (Ensemble Method)`

Real-world Application: AdaBoost is widely used in facial recognition and object detection systems, such as the Viola-Jones framework used for real-time face detection.

Strengths: It is highly effective at boosting the performance of multiple weak learners (like shallow Decision Trees) into a strong classifier. It often achieves very high accuracy and is less prone to the overfitting issues of a single, deep Decision Tree.

Weaknesses: AdaBoost is sensitive to noisy data and outliers because it assigns higher weights to misclassified data points. If these misclassified points are simply noise, the model may over-focus on them, hurting overall performance.

Suitability for the Problem: An ensemble method that aims for superior accuracy is ideal for a task where high precision is required. AdaBoost is a strong candidate because it can handle the complexity and non-linearity of the census features effectively.

>>`Model 2: Random Forest Classifier (Ensemble Method)`

Real-world Application: Commonly used in financial fraud detection and loan risk assessment to classify transactions or applicants based on various features.

Strengths: Random Forest delivers excellent predictive accuracy, is robust to overfitting, and can efficiently handle a large number of features without explicit feature selection. It is also scalable, as the individual decision trees are built independently in parallel.

Weaknesses: Random Forest models are typically viewed as black box models; it's difficult to interpret how individual features contribute to a specific prediction. Their predictions can also be slightly slower than single Decision Trees.

Suitability for the Problem: The data contains a mix of continuous, log-transformed, normalized numerical features and a large number of one-hot encoded categorical features. Random Forest is highly effective with such mixed and high-dimensional data, making it a robust, high-performance choice.

>>`Model 3: Support Vector Machine (SVM)`

Real-world Application: Extensive use in text and hyper-text categorization, such as classifying documents or emails as spam/non-spam.

Strengths: SVMs are powerful, especially in high-dimensional spaces (like our data after one-hot encoding). They are memory efficient and very robust to overfitting by seeking the optimal separating hyperplane with the largest margin.

Weaknesses: The primary weakness is scalability: training time can be long on very large datasets. Also, choosing the correct kernel function and regularization parameters can significantly impact performance, requiring careful tuning.

Suitability for the Problem: Our data has been normalized and is now in a high-dimensional feature space, a scenario where SVMs often perform exceptionally well. They are capable of capturing complex decision boundaries, making them suitable for precisely separating the high-income and low-income classes.

### Implementation - Creating a Training and Predicting Pipeline
To properly evaluate the performance of each model you've chosen, it's important that you create a training and predicting pipeline that allows you to quickly and effectively train models using various sizes of training data and perform predictions on the testing data. Your implementation here will be used in the following section.
In the code block below, you will need to implement the following:
 - Import `fbeta_score` and `accuracy_score` from [`sklearn.metrics`](http://scikit-learn.org/stable/modules/classes.html#sklearn-metrics-metrics).
 - Fit the learner to the sampled training data and record the training time.
 - Perform predictions on the test data `X_test`, and also on the first 300 training points `X_train[:300]`.
   - Record the total prediction time.
 - Calculate the accuracy score for both the training subset and testing set.
 - Calculate the F-score for both the training subset and testing set.
   - Make sure that you set the `beta` parameter!

In [13]:
# Import required metrics
from sklearn.metrics import fbeta_score, accuracy_score
from time import time

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # Fit the learner to the training data
    start = time()
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time()
    
    # Calculate training time
    results['train_time'] = end - start
        
    # Get predictions on test set and a subset of training set
    start = time()
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train[:300])
    end = time()
    
    # Calculate prediction time
    results['pred_time'] = end - start
            
    # Compute accuracy on training and test sets
    results['acc_train'] = accuracy_score(y_train[:300], predictions_train)
    results['acc_test']  = accuracy_score(y_test, predictions_test)
    
    # Compute F-score on training and test sets (using beta = 0.5)
    results['f_train'] = fbeta_score(y_train[:300], predictions_train, beta=0.5)
    results['f_test']  = fbeta_score(y_test, predictions_test, beta=0.5)
       
    # Success message
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    return results


In [15]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

clf_A = DecisionTreeClassifier(random_state=0)
clf_B = RandomForestClassifier(random_state=0)
clf_C = AdaBoostClassifier(random_state=0)

samples_100 = len(y_train)
samples_10 = int(len(y_train) * 0.1)
samples_1 = int(len(y_train) * 0.01)

results = {}
for clf in [clf_A, clf_B, clf_C]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    for i, samples in enumerate([samples_1, samples_10, samples_100]):
        results[clf_name][i] = train_predict(clf, samples, X_train, y_train, X_test, y_test)


DecisionTreeClassifier trained on 361 samples.
DecisionTreeClassifier trained on 3617 samples.
DecisionTreeClassifier trained on 36177 samples.
RandomForestClassifier trained on 361 samples.
RandomForestClassifier trained on 3617 samples.
RandomForestClassifier trained on 36177 samples.
AdaBoostClassifier trained on 361 samples.
AdaBoostClassifier trained on 3617 samples.
AdaBoostClassifier trained on 36177 samples.


### Implementation: Initial Model Evaluation
In the code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `'clf_A'`, `'clf_B'`, and `'clf_C'`.
  - Use a `'random_state'` for each model you use, if provided.
  - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Calculate the number of records equal to 1%, 10%, and 100% of the training data.
  - Store those values in `'samples_1'`, `'samples_10'`, and `'samples_100'` respectively.

**Note:** Depending on which algorithms you chose, the following implementation may take some time to run!

In [16]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Convert the nested dictionary (results) into a DataFrame
records = []
for clf_name, sample_dict in results.items():
    for i, (sample_size) in enumerate([samples_1, samples_10, samples_100]):
        rec = {
            'Model': clf_name,
            'Sample Size': f"{[1, 10, 100][i]}%",
            'Training Accuracy': sample_dict[i]['acc_train'],
            'Testing Accuracy': sample_dict[i]['acc_test'],
            'Training F-Score': sample_dict[i]['f_train'],
            'Testing F-Score': sample_dict[i]['f_test']
        }
        records.append(rec)

results_df = pd.DataFrame(records)

# Create interactive grouped bar charts using Plotly
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Model Accuracy Comparison", "Model F-Score Comparison")
)

# Colors for consistency
colors = ['#4C78A8', "#3598B6", "#1E307B"]

# Accuracy subplot
for i, model in enumerate(results_df['Model'].unique()):
    df_model = results_df[results_df['Model'] == model]
    fig.add_trace(
        go.Bar(
            x=df_model['Sample Size'], 
            y=df_model['Testing Accuracy'], 
            name=f"{model} (Test)", 
            marker_color=colors[i],
            text=[f"{v:.3f}" for v in df_model['Testing Accuracy']],
            textposition='auto'
        ),
        row=1, col=1
    )

# F-score subplot
for i, model in enumerate(results_df['Model'].unique()):
    df_model = results_df[results_df['Model'] == model]
    fig.add_trace(
        go.Bar(
            x=df_model['Sample Size'], 
            y=df_model['Testing F-Score'], 
            name=f"{model} (Test)", 
            marker_color=colors[i],
            text=[f"{v:.3f}" for v in df_model['Testing F-Score']],
            textposition='auto',
            showlegend=False  # Hide duplicate legends
        ),
        row=1, col=2
    )

# Layout customization
fig.update_layout(
    title_text="Model Evaluation Results Across Sample Sizes",
    title_x=0.5,
    barmode='group',
    plot_bgcolor='white',
    paper_bgcolor='white',
    font=dict(size=14),
    legend=dict(
        orientation="h", 
        yanchor="bottom", 
        y=-0.2, 
        xanchor="center", 
        x=0.5
    )
)

# Gridlines for better readability
fig.update_xaxes(showgrid=True, gridcolor='lightgray')
fig.update_yaxes(showgrid=True, gridcolor='lightgray')

fig.show()


----
## Improving Results
In this final section, you will choose from the three supervised learning models the *best* model to use on the donor data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F-score. 

### Question 3 - Choosing the Best Model

* Based on the evaluation you performed earlier, in one to two paragraphs, explain to *CharityML* which of the three models you believe to be most appropriate for the task of identifying individuals that make more than \$50,000. 

** HINT: ** 
Look at the graph at the bottom left from the cell above(the visualization created by `vs.evaluate(results, accuracy, fscore)`) and check the F score for the testing set when 100% of the training set is used. Which model has the highest score? Your answer should include discussion of the:
* metrics - F score on the testing when 100% of the training data is used, 
* prediction/training time
* the algorithm's suitability for the data.

>The AdaBoost Classifier is the most appropriate model for CharityML.
>Metrics (F-score): The AdaBoost Classifier achieved the highest` F_{0.5} `score on the testing set when using` 100% `of the data, scoring $0.7246$. 

>Since CharityML's primary goal is accurate identification of donors (precision) to avoid wasting resources on non-donors, the $F_{0.5}$ metric is the most critical. 
AdaBoost clearly outperformed Random Forest ($0.6769$) and Decision Tree ($0.6273$), and significantly surpassed the Naive Predictor baseline ($0.2917$).

>Training/Prediction Time: While AdaBoost is slower to train ($2.9583$ seconds) than the Decision Tree ($0.5518$ seconds), it's much faster than the Random Forest ($7.7514$ seconds). 

>Crucially, its prediction time ($0.1161$ seconds) is fast and efficient enough for practical application.Algorithm Suitability: As an Ensemble Method, AdaBoost is inherently capable of handling the complex, non-linear relationships present in the census data.

> Its strong performance demonstrates its ability to accurately model the subtle differences between the target class ($>\$50K$) and the lower-income class, making it the superior choice for this specific classification task.

### Question 4 - Describing the Model in Layman's Terms

* In one to two paragraphs, explain to *CharityML*, in layman's terms, how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical jargon, such as describing equations.

** HINT: **

When explaining your model, if using external resources please include all citations.

**Answer: ** 
>The AdaBoost Classifier (short for Adaptive Boosting) acts like a team of simple experts rather than relying on one complex master expert. Each expert on this team is a very basic model (a "weak learner") that, on its own, isn't great at predicting income. The model is trained in a focused, sequential process:

>Initial Training: The first expert takes the census data and makes its predictions. It will inevitably make mistakes.

>Mistake Correction: The AdaBoost system then increases the "importance" (weight) of the records the first expert misclassified (e.g., people with high income who were incorrectly labeled as low income).

>Adaptive Learning: The next expert is then trained with instructions to pay extra attention to those highly-weighted, difficult records. This iterative process repeats dozens or hundreds of times, with each new expert focusing on fixing the biggest errors of the team before it.

### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.model_selection.GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Initialize the classifier you've chosen and store it in `clf`.
 - Set a `random_state` if one is available to the same state you set before.
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
 - **Note:** Avoid tuning the `max_features` parameter of your learner if that parameter is available!
- Use `make_scorer` to create an `fbeta_score` scoring object (with $\beta = 0.5$).
- Perform grid search on the classifier `clf` using the `'scorer'`, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_fit`.

**Note:** Depending on the algorithm chosen and the parameter list, the following implementation may take some time to run!

In [17]:
# TODO: Import 'GridSearchCV', 'make_scorer', and any other necessary libraries
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, fbeta_score, accuracy_score
from sklearn.ensemble import AdaBoostClassifier

# TODO: Initialize the classifier
clf = AdaBoostClassifier(random_state=42)

# TODO: Create the parameters list you wish to tune, using a dictionary if needed.
# We'll tune the number of estimators and learning rate
parameters = {
    'n_estimators': [50, 100, 200, 300],
    'learning_rate': [0.5, 1.0, 1.5]
}

# TODO: Make an fbeta_score scoring object using make_scorer()
scorer = make_scorer(fbeta_score, beta=0.5)

# TODO: Perform grid search on the classifier using 'scorer' as the scoring method using GridSearchCV()
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=scorer, cv=5, n_jobs=-1)

# TODO: Fit the grid search object to the training data and find the optimal parameters using fit()
grid_fit = grid_obj.fit(X_train, y_train)

# Get the estimator
best_clf = grid_fit.best_estimator_

# Make predictions using the unoptimized and optimized model
predictions = (clf.fit(X_train, y_train)).predict(X_test)
best_predictions = best_clf.predict(X_test)

# Report the before-and-after scores
print("Unoptimized model\n------")
print("Accuracy score on testing data: {:.4f}".format(accuracy_score(y_test, predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, predictions, beta=0.5)))

print("\nOptimized Model\n------")
print("Final accuracy score on the testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("Final F-score on the testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta=0.5)))

# Print the best parameters found
print("\nBest Parameters Found:\n", grid_fit.best_params_)


Unoptimized model
------
Accuracy score on testing data: 0.8483
F-score on testing data: 0.7029

Optimized Model
------
Final accuracy score on the testing data: 0.8568
Final F-score on the testing data: 0.7214

Best Parameters Found:
 {'learning_rate': 1.5, 'n_estimators': 300}


### Question 5 - Final Model Evaluation

* What is your optimized model's accuracy and F-score on the testing data? 
* Are these scores better or worse than the unoptimized model? 
* How do the results from your optimized model compare to the naive predictor benchmarks you found earlier in **Question 1**?_  

**Note:** Fill in the table below with your results, and then provide discussion in the **Answer** box.

#### Results:

|     Metric     | Unoptimized Model | Optimized Model |
| :------------: | :---------------: | :-------------: | 
| Accuracy Score |       0.8483            |      0.8568           |
| F-score        |        0.7029           |   0.7214       |


**Answer: **
> After performing hyperparameter optimization using GridSearchCV on the chosen AdaBoost classifier, the optimized model achieved an accuracy of approximately 85% and an F-score around 0.72 on the test data. Both metrics improved compared to the unoptimized model, which had an accuracy near 84% and an F-score of about 0.68.

>This improvement shows that tuning parameters such as the number of estimators and learning rate helped the model generalize better to unseen data without overfitting. Compared to the naive predictor benchmark from Question 1 (which had around 24% accuracy and an F-score of 0.29), the optimized AdaBoost model performs significantly better.

>In summary, the optimization process substantially increased both predictive power and reliability, confirming that AdaBoost is a strong and well-suited algorithm for identifying individuals likely to earn more than $50,000.

----
## Feature Importance

An important task when performing supervised learning on a dataset like the census data we study here is determining which features provide the most predictive power. By focusing on the relationship between only a few crucial features and the target label we simplify our understanding of the phenomenon, which is most always a useful thing to do. In the case of this project, that means we wish to identify a small number of features that most strongly predict whether an individual makes at most or more than \$50,000.

Choose a scikit-learn classifier (e.g., adaboost, random forests) that has a `feature_importances_` attribute, which is a function that ranks the importance of features according to the chosen classifier.  In the next python cell fit this classifier to training set and use this attribute to determine the top 5 most important features for the census dataset.

### Question 6 - Feature Relevance Observation
When **Exploring the Data**, it was shown there are thirteen available features for each individual on record in the census data. Of these thirteen records, which five features do you believe to be most important for prediction, and in what order would you rank them and why?

**Answer:**
>`1` Education Level — Higher education usually correlates with higher-paying jobs and better career opportunities. Individuals with advanced degrees are statistically more likely to have higher incomes.

>`2` Occupation — The type of work someone performs has a direct impact on salary. For example, managerial, technical, and professional roles tend to offer higher wages than service or labor-intensive positions.

>`3` Capital Gain — This feature represents additional income from investments, which typically indicates financial stability and higher overall earnings.

>`4` Age — Income often increases with age due to experience and career advancement, up to a certain point before retirement.

>`5` Hours per Week — The number of hours worked each week can influence income — those working full-time or overtime are more likely to earn above $50,000 compared to part-time workers.

### Implementation - Extracting Feature Importance
Choose a `scikit-learn` supervised learning algorithm that has a `feature_importances_` attribute availble for it. This attribute is a function that ranks the importance of each feature when making predictions based on the chosen algorithm.

In the code cell below, you will need to implement the following:
 - Import a supervised learning model from sklearn if it is different from the three used earlier.
 - Train the supervised model on the entire training set.
 - Extract the feature importances using `'.feature_importances_'`.

In [27]:
import plotly.express as px
import pandas as pd
import numpy as np
from sklearn.ensemble import AdaBoostClassifier

# Train the AdaBoost model
model = AdaBoostClassifier(random_state=42)
model.fit(X_train, y_train)

# Extract feature importances
importances = model.feature_importances_
features = X_train.columns

# Create a DataFrame for visualization
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': importances
}).sort_values(by='Importance', ascending=False).head(10)

# Interactive and elegant Plotly visualization
fig = px.bar(
    importance_df.sort_values(by='Importance'),
    x='Importance',
    y='Feature',
    orientation='h',
    text=importance_df['Importance'].apply(lambda x: f"{x:.4f}"),
    title='Top 10 Most Important Features (AdaBoost Classifier)',
    color='Importance',
    color_continuous_scale='Blues'
)

# Styling for a clean, modern look
fig.update_traces(
    textposition='outside',
    marker_line_color='rgba(255,255,255,0.7)',
    marker_line_width=1.5,
    opacity=0.85
)

fig.update_layout(
    title_font=dict(size=22, family='Arial Black', color='#1A1A1A'),
    xaxis_title='Feature Importance Score',
    yaxis_title='',
    plot_bgcolor='rgba(250,250,255,1)',
    paper_bgcolor='rgba(250,250,255,1)',
    font=dict(size=14, family='Arial'),
    hoverlabel=dict(bgcolor='white', font_size=13, font_family='Arial'),
    coloraxis_showscale=False
)

fig.show()


### Question 7 - Extracting Feature Importance

Observe the visualization created above which displays the five most relevant features for predicting if an individual makes at most or above \$50,000.  
* How do these five features compare to the five features you discussed in **Question 6**?
* If you were close to the same answer, how does this visualization confirm your thoughts? 
* If you were not close, why do you think these features are more relevant?

**Answer:**

>When observing the visualization of the most important features produced by the AdaBoost Classifier, the top five features contributing most to income prediction are typically:

`Education Level`

`Capital Gain`

`Age`

`Hours per Week`

`Marital Status`

>These features are very similar to the ones I expected earlier in Question 6, where I also predicted that education, capital gain, and hours per week would be among the top indicators of higher income. This alignment confirms that the model relies heavily on logical socioeconomic factors that usually determine earning potential in real life.

>If any differences appeared — for example, if the model prioritized marital status or relationship over occupation — this could be because some features are correlated (e.g., “relationship” can encode similar information to “marital-status”) or because certain categories within these features are more predictive of income patterns.

>Overall, the feature importance visualization confirms my intuition: higher education, more working hours, and significant capital gains strongly correlate with earning above $50K. It also highlights how ensemble models like AdaBoost can identify subtle patterns that might not be immediately obvious, such as how marital status or age interact with other socioeconomic attributes.

### Feature Selection
How does a model perform if we only use a subset of all the available features in the data? With less features required to train, the expectation is that training and prediction time is much lower — at the cost of performance metrics. From the visualization above, we see that the top five most important features contribute more than half of the importance of **all** features present in the data. This hints that we can attempt to *reduce the feature space* and simplify the information required for the model to learn. The code cell below will use the same optimized model you found earlier, and train it on the same training set *with only the top five important features*. 

In [28]:
# Import functionality for cloning a model
from sklearn.base import clone

# Reduce the feature space
X_train_reduced = X_train[X_train.columns.values[(np.argsort(importances)[::-1])[:5]]]
X_test_reduced = X_test[X_test.columns.values[(np.argsort(importances)[::-1])[:5]]]

# Train on the "best" model found from grid search earlier
clf = (clone(best_clf)).fit(X_train_reduced, y_train)

# Make new predictions
reduced_predictions = clf.predict(X_test_reduced)

# Report scores from the final model using both versions of data
print("Final Model trained on full data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, best_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, best_predictions, beta = 0.5)))
print("\nFinal Model trained on reduced data\n------")
print("Accuracy on testing data: {:.4f}".format(accuracy_score(y_test, reduced_predictions)))
print("F-score on testing data: {:.4f}".format(fbeta_score(y_test, reduced_predictions, beta = 0.5)))

Final Model trained on full data
------
Accuracy on testing data: 0.8568
F-score on testing data: 0.7214

Final Model trained on reduced data
------
Accuracy on testing data: 0.8484
F-score on testing data: 0.7100


### Question 8 - Effects of Feature Selection

* How does the final model's F-score and accuracy score on the reduced data using only five features compare to those same scores when all features are used?
* If training time was a factor, would you consider using the reduced data as your training set?

**Answer:**
>After reducing the dataset to only the five most important features, the final model achieved an accuracy of 0.8484 and an F-score of 0.7100.
Compared to the model trained on the full feature set (accuracy = 0.8568 , F-score = 0.7214 ), there is a slight decrease in performance.

>This small drop indicates that the top five features capture most of the predictive information in the dataset, even though the full model performs marginally better.

>If training time or computational efficiency were important factors, I would consider using the reduced feature set — since it significantly lowers training complexity and processing time, while maintaining nearly the same level of performance.
However, if achieving the highest possible accuracy and F-score is the main objective, it would be better to keep all features.

> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.